In [ ]:
import openai
import requests
import os
import openai
import json
import logging
from tqdm import tqdm

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.tools import tool

from askem._experimental.testset_ta1 import load_testset

In [ ]:
testset = load_testset().query("is_complex == 1")
testset

Wrap retriever API as `Tool`

In [ ]:
@tool
def search_retriever(query: str) -> str:
    """Useful for when you need to answer questions about facts."""

    RETRIEVER_APIKEY = os.getenv("RETRIEVER_APIKEY")
    RETRIEVER_ENDPOINT = "http://retriever:4502/hybrid"

    headers = {"Content-Type": "application/json", "Api-Key": RETRIEVER_APIKEY}
    data = {
        "question": query,
        "top_k": 3,
        "doc_type": "paragraph",
    }

    response = requests.post(RETRIEVER_ENDPOINT, headers=headers, json=data)
    response.raise_for_status()
    return "\n\n".join([r["text"] for r in response.json()])

Provide retriever search tool to ReAct agent

In [ ]:
llm = OpenAI(model_name="gpt-4", temperature=0)
agent_executor = initialize_agent(
    [search_retriever], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

Tweak summarizer a bit

In [ ]:
def summarize(question: str, contexts: str | list[str]) -> str:
    """Compresses a long text to a shorter version."""
    openai.api_key = os.getenv("OPENAI_API_KEY")
    openai.organization = os.getenv("OPENAI_ORGANIZATION")

    instruction = "Answer the question based on the contexts. If there is no answer in context, say 'no answer'."

    # Provide QA pairs as context
    if isinstance(contexts, str):
        qa_context = [f"{question}: {contexts}"]
    elif isinstance(contexts, list):
        qa_context = [f"{question}: {context}" for context in contexts]
    else:
        raise ValueError(
            f"Contexts must be a string or a list of strings. Got {type(contexts)} instead."
        )

    # Append main question
    print(f"{qa_context=}")
    prompt = f"Question: {question}{os.linesep} Context: {qa_context}"

    print(f"{prompt=}")

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt},
        ],
    )

    return response.choices[0].message.content


def xdd_llm_v02(query: str) -> str:
    context = search_retriever(query)
    summary = summarize(query, context)
    return summary

Testing pipeline

In [ ]:
def _test_one(query: str) -> dict:
    """Compare against past iterations."""

    v02 = xdd_llm_v02(query)
    v03 = agent_executor.invoke({"input": query})["output"]

    return {
        "question": query,
        "v02": v02,
        "v03": v03,
    }


def test(questions: list[str]) -> None:
    """Compare against past iterations."""
    for i, q in tqdm(enumerate(questions)):
        local_result = f"tmp/{i}.json"
        if os.path.exists(local_result):
            continue

        print(f"Testing: {q}")

        result = _test_one(q)
        with open(local_result, "w") as f:
            json.dump(result, f, indent=4)
        # try:
        #     result = _test_one(q)
        #     with open(local_result, "w") as f:
        #         json.dump(result, f, indent=4)
        # except Exception as e:
        #     print(f"Failed: {q}")
        #     print(e)
        #     continue

In [ ]:
test(testset.question.tolist())

Create CSV for sharing

In [ ]:
import pandas as pd
from pathlib import Path

data = []
for x in Path("tmp/").glob("*.json"):
    with open(x) as f:
        data.append(json.load(f))

df = pd.DataFrame(data)

In [ ]:
df.to_csv("tmp/results.csv", index=False)